<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Getting_Hands_On_with_BERT_Fine_Tune_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the dependencies

In [2]:
!pip install nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 46.3 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 


## Importing necessary Modules

In [23]:
from transformers import BertForSequenceClassification,BertTokenizer,BertTokenizerFast, Trainer,TrainingArguments
from nlp import load_dataset
import torch
import pandas as pd
import numpy as np


## Load the dataset

In [7]:
!gdown https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
dataset= load_dataset('csv', data_files='./imdbs.csv', split='train')

Downloading...
From: https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-
To: /content/imdbs.csv
100% 132k/132k [00:00<00:00, 87.0MB/s]


Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-11046c2826f07a01/0.0.0/ede98314803c971fef04bcee45d660c62f3332e8a74491e0b876106f3d99bd9b. Subsequent calls will reuse this data.


In [8]:
type(dataset)

nlp.arrow_dataset.Dataset

In [9]:
dataset

Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 100)

## Split the dataset into train and test sets

In [10]:
dataset = dataset.train_test_split(test_size=0.3)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
dataset

{'train': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 70),
 'test': Dataset(features: {'text': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}, num_rows: 30)}

 ## Create Train and Test Sets

In [12]:
train_set = dataset['train']
test_set = dataset['test']

In [19]:
type(train_set)

nlp.arrow_dataset.Dataset

##Convert nlp.arrow_dataset to pandas DataFrame

In [20]:
pd.DataFrame(train_set)

,label,text
0,0,"If you came here, it's because you've already ..."
1,1,"What an absolutely stunning movie, if you have..."
2,0,"This show was an amazing, fresh & innovative i..."
3,1,"Petter Mattei's ""Love in the Time of Money"" is..."
4,0,"After sitting through this pile of dung, my hu..."
...,...,...
65,0,This was probably the worst movie i have ever ...
66,1,A wonderful little production. <br /><br />The...
67,0,Honestly - this short film sucks. the dummy us...
68,1,I sure would like to see a resurrection of a u...


## Download and load pretrained model

In [21]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Download and load pretained Tokenizer

In [26]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [25]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

## Preprocess Text

In [28]:
 def preprocess(data):
  return tokenizer(data['text'], padding=True, truncation=True)

In [29]:
train_set = train_set.map(preprocess,batched=True,batch_size=len(train_set))
test_set = test_set.map(preprocess,batched=True,batch_size=len(test_set))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
train_set

Dataset(features: {'label': Value(dtype='int64', id=None), 'text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 70)

In [43]:
pd.DataFrame(train_set)


,label,input_ids,attention_mask
0,tensor(0),"[tensor(101), tensor(2065), tensor(2017), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
1,tensor(1),"[tensor(101), tensor(2054), tensor(2019), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
2,tensor(0),"[tensor(101), tensor(2023), tensor(2265), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
3,tensor(1),"[tensor(101), tensor(9004), tensor(3334), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
4,tensor(0),"[tensor(101), tensor(2044), tensor(3564), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
...,...,...,...
65,tensor(0),"[tensor(101), tensor(2023), tensor(2001), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
66,tensor(1),"[tensor(101), tensor(1037), tensor(6919), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
67,tensor(0),"[tensor(101), tensor(9826), tensor(1011), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."
68,tensor(1),"[tensor(101), tensor(1045), tensor(2469), tens...","[tensor(1), tensor(1), tensor(1), tensor(1), t..."


* Can see the input_ids which contains tokens corresponding to the input tokens

## Use the set_format function and select the columns that we need in our dataset and the format we need them in as shown in the following code:

In [34]:
train_set.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])
test_set.set_format('torch',columns=['input_ids', 'attention_mask', 'label'])

## Training the Model

In [36]:
batch_size = 8
epochs = 2
warmup_steps = 500
weight_decay = 0.01

#### Define the training arguments

In [38]:
training_args = TrainingArguments(
          output_dir='./results',
          num_train_epochs=epochs,
          per_device_train_batch_size=batch_size,
          per_device_eval_batch_size=batch_size,
          warmup_steps=warmup_steps,
          weight_decay=weight_decay,
          evaluation_strategy='epoch',
          logging_dir='./logs',
)

https://github.com/huggingface/transformers/issues/9206

## Define Trainer

In [39]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_set,
eval_dataset=test_set
)

## Start training the model:

In [40]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18


Epoch,Training Loss,Validation Loss
1,No log,0.689839
2,No log,0.688773


***** Running Evaluation *****
  Num examples = 30
  Batch size = 8
***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=0.6833469602796767, metrics={'train_runtime': 11.7578, 'train_samples_per_second': 11.907, 'train_steps_per_second': 1.531, 'total_flos': 36835547750400.0, 'train_loss': 0.6833469602796767, 'epoch': 2.0})

In [41]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


{'eval_loss': 0.6887726187705994,
 'eval_runtime': 0.5748,
 'eval_samples_per_second': 52.189,
 'eval_steps_per_second': 6.959,
 'epoch': 2.0}

## Make_Predictions

In [42]:
trainer.predict(test_set)

***** Running Prediction *****
  Num examples = 30
  Batch size = 8


PredictionOutput(predictions=array([[ 0.16253442,  0.014405  ],
       [ 0.25181094,  0.02531632],
       [ 0.19457656,  0.06417143],
       [ 0.25377777,  0.07177623],
       [ 0.40345392,  0.21447887],
       [ 0.25635186,  0.07956313],
       [ 0.05320898, -0.14229836],
       [ 0.15464918, -0.04394291],
       [ 0.46617588,  0.32382908],
       [ 0.5762367 ,  0.33819845],
       [ 0.08957136, -0.01002837],
       [ 0.45991528,  0.24666332],
       [ 0.27185056,  0.06413858],
       [ 0.17617324,  0.12030941],
       [ 0.49723735,  0.33013186],
       [ 0.21400173,  0.02479028],
       [ 0.27525064,  0.11396804],
       [ 0.34290192,  0.08475005],
       [ 0.34016863,  0.1325411 ],
       [ 0.29794368,  0.06621175],
       [ 0.3142627 ,  0.06723689],
       [ 0.20674695,  0.07067182],
       [ 0.4627678 ,  0.3081502 ],
       [ 0.24021827,  0.0498436 ],
       [ 0.2685578 ,  0.18572338],
       [ 0.27820393,  0.12069681],
       [-0.02699796, -0.08510757],
       [ 0.20924395, -0.03

In [44]:
predictions = trainer.predict(test_set)

***** Running Prediction *****
  Num examples = 30
  Batch size = 8


In [48]:
predictions.predictions

array([[ 0.16253442,  0.014405  ],
       [ 0.25181094,  0.02531632],
       [ 0.19457656,  0.06417143],
       [ 0.25377777,  0.07177623],
       [ 0.40345392,  0.21447887],
       [ 0.25635186,  0.07956313],
       [ 0.05320898, -0.14229836],
       [ 0.15464918, -0.04394291],
       [ 0.46617588,  0.32382908],
       [ 0.5762367 ,  0.33819845],
       [ 0.08957136, -0.01002837],
       [ 0.45991528,  0.24666332],
       [ 0.27185056,  0.06413858],
       [ 0.17617324,  0.12030941],
       [ 0.49723735,  0.33013186],
       [ 0.21400173,  0.02479028],
       [ 0.27525064,  0.11396804],
       [ 0.34290192,  0.08475005],
       [ 0.34016863,  0.1325411 ],
       [ 0.29794368,  0.06621175],
       [ 0.3142627 ,  0.06723689],
       [ 0.20674695,  0.07067182],
       [ 0.4627678 ,  0.3081502 ],
       [ 0.24021827,  0.0498436 ],
       [ 0.2685578 ,  0.18572338],
       [ 0.27820393,  0.12069681],
       [-0.02699796, -0.08510757],
       [ 0.20924395, -0.03140412],
       [ 0.23715474,

In [49]:
predictions.label_ids

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1])

In [50]:
predictions.metrics

{'test_loss': 0.6887726187705994,
 'test_runtime': 0.604,
 'test_samples_per_second': 49.671,
 'test_steps_per_second': 6.623}